# Draft of a ZNE toolkit

Actually this notebook is self-consistent and does not import any toolkit. This is useful for understanding and prototyping the main components of the toolkit.

In [107]:
# IBM SDK
import qiskit
from qiskit import QuantumCircuit

# Noise simulation packages
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error

# Other tools
import numpy as np
%matplotlib inline
from matplotlib import pylab as plt

The following function is not related to the toolkit. We just use it to generate random circuits.

In [108]:
def random_identity_circuit(depth=None):
    """Returns a single-qubit identity circuit based on Pauli gates."""
    
    # initialize a quantum circuit with 1 qubit and 1 classical bit
    circuit = QuantumCircuit(1, 1)
    
    # index of the (inverting) final gate: 0=I, 1=X, 2=Y, 3=Z
    k_inv = 0

    # apply a random sequence of Pauli gates
    for _ in range(depth):
        # random index for the next gate: 1=X, 2=Y, 3=Z
        k = np.random.choice([1, 2, 3])
        # apply the Pauli gate "k"
        if k == 1:
            circuit.x(0)
        elif k == 2:
            circuit.y(0)
        elif k == 3:
            circuit.z(0)
        
        # update the inverse index according to 
        # the product rules of Pauli matrices k and k_inv
        if k_inv == 0:
            k_inv = k
        elif k_inv == k:
            k_inv = 0
        else:
            _ = [1, 2, 3]
            _.remove(k_inv)
            _.remove(k)
            k_inv = _[0]
    
    # apply the final inverse gate
    if k_inv == 1:
        circuit.x(0)
    elif k_inv == 2:
        circuit.y(0)
    elif k_inv == 3:
        circuit.z(0)
        
    return circuit

## We focus on the simplest scenario: we assume the user can increase the noise level

We assume the user can define a function which evaluates an input circuit at a given noise level and returns a single expectation value. For example:

In [117]:
def circ_to_expval(circuit=None, stretch=1):
    """Executes a circuit on a noisy device and returns the expectation value of 
    the final measurement. The function is not expected/required to be well defined for stretch<1,
    i.e., noise can be increased but not reduced. """
    
    shots = 10 ** 5       # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch
    
    # initialize a qiskit noise model
    noise_model = NoiseModel()
    
    # we assume a depolarizing error for each gate of the standard IBM basis set (u1, u2, u3)
    noise_model.add_all_qubit_quantum_error(depolarizing_error(noise, 1), ['u1', 'u2', 'u3'])
    
    # execution of the experiment
    job = qiskit.execute(circuit, 
                         backend=backend, 
                         basis_gates=['u1', 'u2', 'u3'],
                         # we want all gates to be actually applied,
                         # so we skip any circuit optimization
                         optimization_level=0, 
                         noise_model=noise_model,
                         shots=shots)
    results = job.result()
    counts = results.get_counts()
    expval = counts['0'] / shots
    return expval

Let us test the function above for different levels of noise

In [118]:
# beckend initialization
backend = qiskit.Aer.get_backend('qasm_simulator')

# generate a random circuit
circuit = random_identity_circuit(depth=100)

# add a final measurement in the z basis, ideally 1 without noise.
circuit.measure(0, 0)

for j in range(5):
    expval = circ_to_expval(circuit=circuit, stretch=j)
    note = ""
    if j == 0:
        note = "(noiseless result not accessible to the user in a real scenario)"
    if j == 1:
        note = "(result without error mitigation)"
    print("stretch:{}  expval:{:.2f} {}".format(j, expval, note))

stretch:0  expval:1.00 (noiseless result not accessible to the user in a real scenario)
stretch:1  expval:0.75 (result without error mitigation)
stretch:2  expval:0.62 
stretch:3  expval:0.56 
stretch:4  expval:0.53 


## The Mitigator object

First, we define two auxiliary functions:

In [111]:
def get_gammas(c):
    """Returns the linear combination coefficients "gammas" for Richardson's extrapolation.
    The input is a list of the noise stretch factors.    
    """
    order = len(c) - 1
    np_c = np.asarray(c)
    A = np.zeros((order + 1, order + 1))
    for k in range(order + 1):
        A[k] = np_c ** k
    b = np.zeros(order + 1)
    b[0] = 1
    return np.linalg.solve(A, b) 

In [112]:
def check_c(order, c):
    """Consistency check of the noise stretch vector. Returns c[:order + 1]. If c is None, generates a default one."""
    if c == None:
        # generate a default list
        c = list(range(1, order + 2))
    if order > len(c) - 1:
        raise ValueError("Extrapolation order is too high compared to len(c) - 1.") 
    if c[0] != 1:
        raise ValueError("c[0] should be 1.") # Not sure if this is really a requirement.
    return c[:order + 1]

Now we can define a key component of the toolkit: the `Mitigator` class.

In [113]:
class Mitigator:
    """"Error mitigation class. 
    It can be used to process a quantum circuit in order to extrapolate an error-mitigated result.
    """
        
    def __init__(self, circ_to_expval, order=0, c=None, method='richardson'):
        """Initializes the mitigator."""
        # consistency check of the arguments
        check_c(order, c)
        # initialize object variables
        self.circuit = None
        self.circ_to_expval = circ_to_expval
        self.order = order
        self.c = c
        self.method = method
        self.expvals = []
        self.result = None
    
    def load(self, circuit):
        """Loads the circuit into the mitigator object."""
        self.circuit = circuit
    
    def comp(self):
        """Compiles the circuit for error mitigation purposes."""
        # if the user is able to control the noise, do nothing.
        pass
    
    def run(self):
        """Executes the circuit for different noise levles"""
        _c = check_c(self.order, self.c)
        self.expvals = []
        for j, c_val in enumerate(_c):
            self.expvals.append(self.circ_to_expval(circuit=self.circuit, stretch=c_val))
        return self.expvals
    
    def extrapolate(self):
        """Extrapolates from the list self.expvals"""
        if self.method == 'richardson':
            _c = check_c(self.order, self.c)
            # get linear combination coefficients
            gammas = get_gammas(_c)
            # linear extraolation
            self.result = np.dot(gammas, self.expvals[0:self.order + 1])
        return self.result
    
    def __call__(self, circuit):
        """Evaluates the expectation value of the input circuit with error mitigation"""
        self.load(circuit)
        self.comp()
        self.run()
        self.extrapolate()
        return self.result
        
    

## Manually using a *Mitigator* object  (no *decorator*)

The user can instantiate a mitigator object in this way:

In [114]:
mit = Mitigator(circ_to_expval, order=3)

Assume that we want to evaluate the following random circuit:

In [115]:
rand_circ = random_identity_circuit(depth=100)
rand_circ.measure(0, 0)

There are different ways of applying the mitigator `mit` to the circuit `rand_circ`.

### Step by step processing

* Load the circuit into the mitigator:

In [99]:
mit.load(rand_circ)

* Compile (if necessary) and execute the circuit at different noise levels:

In [100]:
mit.comp()
mit.run()

[0.74598, 0.61994, 0.55872, 0.52864]

* Extrapolate to the zero noise limit:

In [101]:
mit.extrapolate()

0.9705199999999996

### Direct processing of the mitigated result

Alternatively, the user can directly get the mitigated result as follows:

In [103]:
mit(rand_circ)

0.9628099999999999

**Note:** Different evaluations of the mitigator produce slightly different results because of the finite number of shots.

## Directly *decorating* user's functions

Let us define a Python *decorator* (`mitigate`) associated to the class `Mitigator`:

In [104]:
# more precisely, this is a wrap function which is necessary to pass some parameters to the decorator
def mitigate(order=0, c=None, method='richardson'):
    
    # formally, this is the actual decorator
    def create_mitigator(fun):
        return Mitigator(fun, order=order, c=c, method=method)
    
    return create_mitigator   

The user can directly decorate its own functions with a single line of code `@mitigate()`, to be placed right above the function definition. For example:

In [105]:
@mitigate(order=3)
def magic_run(circuit=None, stretch=1):
    """Execute a circuit on a noisy device and returns the expectation value of 
    the final measurement."""
    
    shots = 10 ** 5       # measurement shots
    true_noise = 0.007    # real value of the noise
    
    # noise stretching
    noise = true_noise * stretch
    
    # initialize a qiskit noise model
    noise_model = NoiseModel()
    
    # we assume a depolarizing error for each gate of the standard IBM basis set (u1, u2, u3)
    noise_model.add_all_qubit_quantum_error(depolarizing_error(noise, 1), ['u1', 'u2', 'u3'])
    
    # execution of the experiment
    job = qiskit.execute(circuit, 
                         backend=backend, 
                         basis_gates=['u1', 'u2', 'u3'],
                         # we want all gates to be actually applied,
                         # so we skip any circuit optimization
                         optimization_level=0, 
                         noise_model=noise_model,
                         shots=shots)
    results = job.result()
    counts = results.get_counts()
    expval = counts['0'] / shots
    return expval

**Note:** The body of the function `magic_run` is a the same as the previous `circ_to_expval`. We simply used a different name to avoid conflicts with the previous part of this notebook.

Now the user can directly apply his/her own function `magic_run` to evaluate circuits. Errors will be magically mitigated behind the scenes!

In [106]:
magic_run(rand_circ)

0.96571

## Advanced usage

Because of the decorator, `magic_run` is actually not a function but an instance of the `Mitigator` class. This means that, if necessary, we can access some of its attributes. For example, we can change the extrapolation order:

In [87]:
magic_run.order=0
magic_run(rand_circ)

0.74632

In [88]:
magic_run.order=4
magic_run(rand_circ)

0.9869300000000009

Another useful possibility is that of visualizing and/or manually modifying the internal list of expectation values `self.expvals`:

In [89]:
expv = magic_run.expvals
# add some random shifts to the internal list of expectation values
magic_run.expvals = [expv[j] + 0.02 * np.random.rand() for j in range(len(expv))]
# extrapolate
magic_run.extrapolate()

0.8633593474261136

**Note:** running the previous cell many times one can classically estimate the statistical error without performing additional quantum experiments (*bootstrapping*).